In [ ]:
# import libraries
import requests
import pandas as pd
import json
from datetime import datetime

# CityBikes

Send a request to CityBikes for the city of your choice. 

In [ ]:
# function to make api request
def get_networks():
    url = "http://api.citybik.es/v2/networks"
    response = requests.get(url)
    networks = response.json()
    return networks

networks = get_networks()

# create a dataframe from the networks data
networks_df = pd.DataFrame(networks['networks'])

# find 'Vancouver' in the networks data
# iterate over the rows of the DataFrame
for index, row in networks_df.iterrows():
    # check if the 'city' key in the 'location' dictionary is 'Vancouver'
    if row['location']['city'] == 'Vancouver':
        vancouver_network = row

# make the API call to get detailed network data for vancouver, using href found in above vancouver network data
network_detail_url = f"http://api.citybik.es{vancouver_network['href']}"
response = requests.get(network_detail_url)
van_network_data = response.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [70]:
# put the data into a DataFrame
van_network_df = pd.DataFrame(van_network_data)

# get the stations data from the DataFrame
van_stations_df = pd.DataFrame(van_network_df.loc['stations'][0])

In [ ]:
# save raw citybikes response data to json to avoid repeat api calls
van_stations_df.to_json('../data/citybikes_response.json')

In [71]:
# format and filter dataframe

# Convert to datetime with UTC timezone
van_stations_df['timestamp'] = pd.to_datetime(van_stations_df['timestamp'], utc=True)
# Convert from UTC to local time (Vancouver)
van_stations_df['timestamp'] = van_stations_df['timestamp'].dt.tz_convert('US/Pacific')
# Split timestamp into date and time columns
van_stations_df['date'] = pd.to_datetime(van_stations_df['timestamp']).apply(lambda x: x.strftime("%Y-%m-%d"))
van_stations_df['time'] = pd.to_datetime(van_stations_df['timestamp']).apply(lambda x: x.strftime("%H:%M:%S"))

# flatten the 'extra' column into into separate columns
van_stations_df = pd.concat([van_stations_df.drop(['extra'], axis=1), van_stations_df['extra'].apply(pd.Series)], axis=1)

# drop redundant/unneeded columns (has ebikes is always True)
van_stations_df = van_stations_df.drop(['has_ebikes', 'uid', 'timestamp', 'last_updated'], axis=1)

# rename columns for clarity
van_stations_df.rename(columns={
    'name': 'station_name', 'id': 'station_id', 'latitude': 'station_latitude', 
    'longitude': 'station_longitude', 'slots': 'total_slots', 
    'free_bikes': 'total_bikes_available', 'ebikes': 'ebikes_available',
    'normal_bikes': 'normal_bikes_available', 'date': 'station_info_date',
    'time': 'station_info_time', 'renting': 'can_rent', 'returning': 'can_return'
}, inplace=True)

# reorder columns prioritizing data of interest
van_stations_df = van_stations_df[[
    'station_name', 'total_slots', 'empty_slots', 'total_bikes_available', 
    'normal_bikes_available', 'ebikes_available', 'station_info_date',
    'station_info_time', 'station_latitude', 'station_longitude', 'can_rent', 
    'can_return', 'station_id'
]]


Put your parsed results into a DataFrame.

In [72]:
# save the dataframe to json for use in yelp_foursqure.ipynb
van_stations_df.to_json('../data/citybikes_data.json')

In [73]:
van_stations_df.head()

,station_name,total_slots,empty_slots,total_bikes_available,normal_bikes_available,ebikes_available,station_info_date,station_info_time,station_latitude,station_longitude,can_rent,can_return,station_id
0,10th & Cambie,36,13,22,17,5,2023-12-09,13:39:21,49.262487,-123.114397,True,True,7a19c49f486d7c0c02b3685d7b240448
1,Yaletown-Roundhouse Station,16,11,5,4,1,2023-12-09,13:39:21,49.274566,-123.121817,True,True,32603a87cfca71d0f7dfa3513bad69d5
2,Dunsmuir & Beatty,26,1,25,21,4,2023-12-09,13:39:21,49.279764,-123.110154,True,True,6d42fa40360f9a6b2bf641c7b8bb2862
3,12th & Yukon (City Hall),16,13,2,1,1,2023-12-09,13:39:21,49.260599,-123.113504,True,True,66f873d641d448bd1572ab086665a458
4,8th & Ash,16,9,7,2,5,2023-12-09,13:39:21,49.264215,-123.117772,True,True,485d4d24c803cfde829ab89699fed833
